<h1>Casestudy Eric Bühler - Aachen </h1>

In [3]:
# Packages used in this notebook
import requests
from nltk.sentiment import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup
import collections
from datetime import datetime, timedelta
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
import nltk
import numpy as np
import pandas as pd
import configparser
import openai


SCRAPER_API_KEY = '33141f1d2afa5cf3c5dfe67546f6aa8b'  # Replace with your Scraper API key

def get_wg_links(pages=2):
    links = []
    for i in range(pages):
        url = f"https://www.wg-gesucht.de/wg-zimmer-in-Aachen.1.0.1.{i}.html?offer_filter=1&city_id=1&sort_order=0&noDeact=1&categories%5B%5D=0&pagination=1&pu="
        scraper_api_url = f"http://api.scraperapi.com?api_key={SCRAPER_API_KEY}&url={url}"
        response = requests.get(scraper_api_url)
        
        if response.status_code != 200:
            print(f"Failed to retrieve {url}, status code: {response.status_code}")
            print(response.text)
            continue

        # Parse results
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extracting all Anzeigen
        anzeigen_raw = soup.find_all('div', class_='col-sm-8 card_body')
        for anzeige in anzeigen_raw:
            anchor_tag = anzeige.find('a')
            if anchor_tag and 'href' in anchor_tag.attrs:
                links.append("https://www.wg-gesucht.de" + anchor_tag['href'])

    return links

def get_anzeigen_html(links):
    anzeigen_html = []
    for link in links:
        scraper_api_url = f"http://api.scraperapi.com?api_key={SCRAPER_API_KEY}&url={link}"
        response = requests.get(scraper_api_url)
        
        if response.status_code != 200:
            print(f"Failed to retrieve {link}, status code: {response.status_code}")
            print(response.text)
            continue

        # Store the BeautifulSoup object
        soup = BeautifulSoup(response.text, 'html.parser')
        anzeigen_html.append({'html':soup,'link':link})
    return pd.DataFrame(anzeigen_html)

def analyze_sentiment(article_text):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(article_text)
    return sentiment

def calculate_composite_sentiment(zimmer_text,lage_text,wg_leben_text,sonstiges_text):
    texts = {
        'zimmer': zimmer_text,
        'lage': lage_text,
        'wg_leben': wg_leben_text,
        'sonstiges': sonstiges_text
    }

    sentiments = {}
    valid_compound_scores = []

    for key, text in texts.items():
        if text != 'Unknown':
            sentiment = analyze_sentiment(text)
            sentiments[key] = sentiment
            valid_compound_scores.append(sentiment['compound'])
        else:
            sentiments[key] = {'compound': None, 'neg': None, 'neu': None, 'pos': None}

    if valid_compound_scores:
        overall_sentiment = sum(valid_compound_scores) / len(valid_compound_scores)
    else:
        overall_sentiment = None

    sentiments['overall_sentiment'] = overall_sentiment

    return sentiments

def get_anzeigen_from_html(html_list):
    anzeigen = []
    for soup in html_list['html']:
        try:
            titel_element = soup.find('h1', class_='headline headline-detailed-view-title')
            titel = titel_element.get_text().replace('\n', '').strip() if titel_element else 'Unknown'

            wg_groesse_element = soup.find('span', class_='mr5')
            wg_groesse = wg_groesse_element['title'].split('er')[0].split()[0] if (wg_groesse_element and 'title' in wg_groesse_element.attrs) else 'Unknown'

            key_facts = soup.find_all('b', class_='key_fact_value')
            groesse = key_facts[0].get_text().split('m')[0].strip() if key_facts and len(key_facts) > 0 else 'Unknown'
            miete = key_facts[1].get_text().split('€')[0].strip() if key_facts and len(key_facts) > 1 else 'Unknown'

            minor_facts = soup.find_all('span', class_='section_panel_value')
            kaltmiete = minor_facts[0].get_text().replace('€','').strip() if minor_facts and len(minor_facts) > 0 else 'Unknown'
            nebenkosten = minor_facts[1].get_text().replace('€','').strip() if minor_facts and len(minor_facts) > 1 else 'Unknown'
            sonstige_kosten = minor_facts[2].get_text().replace('€','').strip() if minor_facts and len(minor_facts) > 2 else 'Unknown'
            kaution = minor_facts[3].get_text().replace('€','').strip() if minor_facts and len(minor_facts) > 3 else 'Unknown'
            abloesevereinb = minor_facts[4].get_text().replace('€','').strip() if minor_facts and len(minor_facts) > 4 else 'Unknown'
            frei_ab = minor_facts[5].get_text().strip() if minor_facts and len(minor_facts) > 5 else 'Unknown'

            adresse = soup.find_all('a', href='#mapContainer')
            strasse = adresse[0].get_text().strip().split()[0] if adresse else 'Unknown'
            plz = next((text for text in adresse[0].get_text().strip().split() if re.fullmatch(r'\d{5}', text)), 'Unknown') if adresse else 'Unknown'

            online_seit = soup.find('b', style='color: #218700;').get_text().strip() if soup.find('b', style='color: #218700;') else 'Unknown'
         
            if "Sekunde" in online_seit or "Sekunden" in online_seit:
                online_seit = "00:00"
            elif "Minuten" in online_seit or "Minute" in online_seit:
                minutes = ''.join(filter(str.isdigit, online_seit))
                online_seit = f"00:{minutes.zfill(2)}"
            elif "Stunden" in online_seit or "Stunde" in online_seit:
                hours = ''.join(filter(str.isdigit, online_seit))
                online_seit = f"{hours.zfill(2)}:00"
            elif "Tag" in online_seit or "Tage" in online_seit:
                days = ''.join(filter(str.isdigit, online_seit))
                hours = int(days)*24
                online_seit = f"{hours}:00"

            zimmer = soup.find('div', id='freitext_0')
            zimmer = zimmer.find('p').get_text().strip() 
            zimmer = re.sub('[^a-zA-ZäöüÄÖÜß-]', ' ', zimmer) if zimmer else 'Unknown'

            lage = soup.find('div', id='freitext_1')
            lage = lage.find('p').get_text().strip() 
            lage = re.sub('[^a-zA-ZäöüÄÖÜß-]', ' ', lage) if lage else 'Unknown'

            wg_leben = soup.find('div', id='freitext_2')
            wg_leben = wg_leben.find('p').get_text().strip() 
            wg_leben = re.sub('[^a-zA-ZäöüÄÖÜß-]', ' ', wg_leben) if wg_leben else 'Unknown'

            sonstiges = soup.find('div', id='freitext_3')
            sonstiges = sonstiges.find('p').get_text().strip()
            sonstiges = re.sub('[^a-zA-ZäöüÄÖÜß-]', ' ', sonstiges) if sonstiges else 'Unknown'
        
            sentiment=calculate_composite_sentiment(zimmer,lage,wg_leben,sonstiges)['overall_sentiment']

            link = html_list['link'].iloc[0]

            anzeigen.append({
                'titel': titel,
                'bewohner': wg_groesse,
                'groesse': groesse,
                'miete': miete,
                'strasse': strasse,
                'plz': plz,
                'online_seit': online_seit,
                'sentiment': sentiment,
                'zimmer': zimmer,
                'lage': lage,
                'wg_leben': wg_leben,
                'sonstiges': sonstiges,
                'link': link
                
            })

        except Exception as e:
            print(f"An error occurred: {e}, link: {html_list['link'].iloc[0]}")
            continue
            # Specify the file path
    file_path = 'anzeigen.csv'

    # Save DataFrame to CSV
    pd.DataFrame(anzeigen).to_csv(file_path, index=False, encoding='utf-8')

    return pd.DataFrame(anzeigen)


links = get_wg_links()
anzeigen_html = get_anzeigen_html(links)
anzeigen = get_anzeigen_from_html(anzeigen_html)
display(anzeigen)


An error occurred: 'NoneType' object has no attribute 'find', link: https://www.wg-gesucht.de/wg-zimmer-in-Aachen-Aachen.6839307.html
An error occurred: 'NoneType' object has no attribute 'find', link: https://www.wg-gesucht.de/wg-zimmer-in-Aachen-Aachen.6839307.html
An error occurred: 'NoneType' object has no attribute 'find', link: https://www.wg-gesucht.de/wg-zimmer-in-Aachen-Aachen.6839307.html


,titel,bewohner,groesse,miete,strasse,plz,online_seit,sentiment,zimmer,lage,wg_leben,sonstiges,link
0,Möbliertes Zimmer in Uninähe,11,21,265,Salvatorstr.,52070,01:00,-0.881325,Du studierst an der RWTH-Aachen und vorzugswei...,Das Haus befindet sich in fußläufiger Nähe zum...,Wir sind die Aachener Burschenschaft Teutonia ...,Da wir täglich viele Anfragen bekommen freut ...,https://www.wg-gesucht.de/wg-zimmer-in-Aachen-...
1,Newly Renovated,3,11,455,Schlosspark,52072,01:00,0.489825,Die frisch renovierte -Zimmer-WG befindet sic...,Eine Bushaltestelle befindet sich direkt vor d...,Ich bin Younes und mein Zimmer wäre deins A...,Weitere Merkmale des Zimmers und der Wohnung s...,https://www.wg-gesucht.de/wg-zimmer-in-Aachen-...
2,3-Flatshared Apartment,3,11,455,Schlosspark,52072,01:00,0.502825,Das Zimmer befindet sich in einer frisch renov...,Die ruhige und gelassene Nachbarschaft ist ein...,Zurzeit wohnen Atif ein Softwareentwickler u...,Weitere Merkmale des Zimmers und der Wohnung s...,https://www.wg-gesucht.de/wg-zimmer-in-Aachen-...
3,"14m2 Zimmer in ""Doppel-4er-WG"" - Haus mit Gart...",4,14,430,Muffeter,52074,02:00,-0.388250,In unserer gemütlichen Doppel er-WG zwei ...,Das Haus liegt in bester und ruhiger Lage au...,Wir sind eine bunt gemischte und gesellige WG ...,Im Preis sind Miete Internet Strom Wasser ...,https://www.wg-gesucht.de/wg-zimmer-in-Aachen-...
4,WG Zimmer in moderner Wohnung und zentraler Lage,3,14,450,Normaluhr,52064,02:00,-0.882275,Das zu vermietende Zimmer ist ca Quadratme...,Die Wohnung liegt zentral an der Kreuzung Norm...,Dein Vormieter Jannik verlässt uns leider da ...,In deinem zukünftigen Zimmer würde das Hochbet...,https://www.wg-gesucht.de/wg-zimmer-in-Aachen-...
5,WG-Zimmer in zentraler Lage in der Aachener In...,3,16,375,Krakaustraße,52064,02:00,-0.488550,Hallo zusammen Ich Kamil ziehe zum ...,Die Lage im Jakobsviertel ist wirklich top Bi...,Du würdest mit Marc und U ur zusammenwohnen ...,Wenn du dich angesprochen fühlst und Lust auf ...,https://www.wg-gesucht.de/wg-zimmer-in-Aachen-...
6,🏘 girls only apartment /rooms for female stude...,3,20,480,monheimsalle,52062,06:00,0.605800,Free room in a shared WG -rooms apartments ...,Best location of Aachen,Amazing apartment,All dAy flexibility,https://www.wg-gesucht.de/wg-zimmer-in-Aachen-...
7,Großes Zimmer in entspannter 5er WG mit Katze,5,19,419,Aretzstraße,52070,09:00,-0.652925,Dein Zimmer wird eins der größten der WG sein ...,Die WG liegt direkt am Europaplatz Blücherplat...,Liz und Till ziehen leider beide zum Septe...,Da zwei Leute ausziehen gibt es auch zwei Zimm...,https://www.wg-gesucht.de/wg-zimmer-in-Aachen-...
8,"Helles, freundliches WG-Zimmer zur Zwischenmie...",3,16,300,Bosstraat,Unknown,10:00,0.006050,English below Auch für kürzere Unterm...,Das Zimmer ist in Vaals gelegen einem Vorort ...,Du wirst zusammen mit Yoana und Simone wohnen ...,Da ich bereits ausgezogen bin würde ich das Z...,https://www.wg-gesucht.de/wg-zimmer-in-Aachen-...
9,Neuer/e Mitbewohner/in in 2er WG gesucht,2,11,318,Alsenstraße,52068,10:00,-0.253700,Please note the english translation below ...,German Netto und Bushaltestelle sind direkt...,German Ich heiße Chinonso ich bin Fachinfo...,WG Ausstattung ist folgendermaßen Laminat ...,https://www.wg-gesucht.de/wg-zimmer-in-Aachen-...


In [21]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


anzeigen=np.asarray(anzeigen)
# Assuming 'anzeigen' DataFrame is already prepared with relevant data

# Convert 'plz' to categorical and create dummy variables
anzeigen['plz'] = anzeigen['plz'].astype('category')
plz_dummies = pd.get_dummies(anzeigen['plz'], drop_first=True, prefix='plz')

# Concatenate dummy variables with original DataFrame
anzeigen = pd.concat([anzeigen, plz_dummies], axis=1)

# Prepare X (independent variables) and y (dependent variable)
X = anzeigen[['bewohner', 'groesse'] + list(plz_dummies.columns)]
X = X.apply(pd.to_numeric, errors='coerce')  # Convert all columns to numeric

y = pd.to_numeric(anzeigen['miete'], errors='coerce')  # Ensure 'miete' is numeric

# Clean NaN values if present
X = X.dropna()
y = y[X.index]  # Ensure y matches the cleaned X indices

# Add intercept to X
X = sm.add_constant(X)

# Fit OLS regression model
model = sm.OLS(y, X)
results = model.fit()

# Print summary of regression results
print(results.summary())


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices